## [실습] 데이터 전처리 : 카카오톡 대화방 데이터

1. 데이터 준비
2. 데이터 전처리
- [참고] https://wikidocs.net/141389

------------------------

### 1. 데이터 준비하기 

#### (모바일에서)
1. 분석할 특정 채팅방 선택하기
2. 화면 우측 상단 더보기(줄세개) 선택하기
3. 화면 우측 하단 채팅방 설정(톱니바퀴) 선택하기
4. 대화 내용 내보내기 선택 (텍스트 메시지만 보내기)
5. 메일 등을 이용하여 다운로드 받아 data 폴더로 이동시키기


------

### 2. 데이터 전처리 (데이터 파싱하기)
판다스로 csv 파일 불러와 확인하기

#### 2.1 데이터의 컬럼(feature) 살펴보기

#### AndroidPhone 내용

In [ ]:
import pandas as pd

# 자신의 카카오톡 데이터 파일명으로 변경한다.
file = '  '
df = pd.read_table(file)
df.head()

#### 2.2  카카오톡 데이터 파싱하기
- 참고:  https://regexr.com/ 

[예제 문장] 2023년 3월 22일 오전 11:36, 송혜경 : 이모티콘 (Android)

1. 카카오톡 데이터 쪼개기
    - 날짜와 시간: (iOS: . / Android: 년,월,일
    - 보낸 사람과 텍스트
2. 카카오톡 데이터 파싱하기
    - 정규표현식 매칭 & 메시지 파싱
3. 데이터 프레임으로 만들기

In [ ]:
import re
import pandas as pd
import os

def katalk_msg_parse(file_path):
    my_katalk_data = list()
    
    # 정규 패턴 정의
    katalk_msg_pattern = "[0-9]{4}[년.] [0-9]{1,2}[월.] [0-9]{1,2}[일.] 오\S [0-9]{1,2}:[0-9]{1,2},.*:"
    date_info = "[0-9]{4}년 [0-9]{1,2}월 [0-9]{1,2}일 \S요일"
    in_out_info = "[0-9]{4}[년.] [0-9]{1,2}[월.] [0-9]{1,2}[일.] 오\S [0-9]{1,2}:[0-9]{1,2}:.*"

    # 파일 데이터 읽기
    cnt = 0
    f = open(file_path, encoding='utf-8')    
    for line in f :
        cnt += 1
        if re.match(date_info, line) or re.match(in_out_info, line):
            continue
        elif line == '\n': # 엔터
            continue
        elif re.match(katalk_msg_pattern, line):          # 카카오톡 메시지 패턴
            line = line.split(",")
            date_time = line[0]                           # 년월일 시간
            user_text = line[1].split(" : ", maxsplit=1)  
            user_name = user_text[0].strip()              # 메시지 보낸 사람
            text = user_text[1].strip()                   # 메시지
            # DataFrame으로 나타내기 위해 my_katalk_data 리스트에 딕셔너리 타입으로 추가하기
            my_katalk_data.append({'date_time': date_time,
                                   'user_name': user_name,
                                   'text': text})
        else:
            if len(my_katalk_data) > 0:
                my_katalk_data[-1]['text'] += "\n"+line.strip()
    print(f'파일 read count: {cnt}')
    
    # DataFrame 데이터로 만들기
    my_katalk_df = pd.DataFrame(my_katalk_data)
    print(f'DataFrame count: {len(my_katalk_df)}')
    return my_katalk_df


df = katalk_msg_parse(file)

df.head()

#### iOS 내용

In [ ]:
# 저장한 날짜 : 2023-06-11 16:01:36 

# --------------- 2023년 3월 21일 화요일 ---------------
# user1님이 user2님을 초대하였습니다.
# [user1] [오전 11:22] 반갑습니다 :) 저는 제니퍼입니당!
# [user1] [오전 11:22] 이모티콘
# [user2] [오전 11:23] 안녕하세요!! 그레이스입니다 :))
# [user2] [오전 11:23] 잘부탁드려용
# [user3] [오전 11:23] 반갑습니다~ 저희도 잘 부탁 드립니다~!

In [ ]:
import re
import pandas as pd
import os

# 자신의 카카오톡 데이터 파일명으로 변경한다.
file = ' '
df = pd.read_table(file)
df

def katalk_msg_parse_iphone(file_path):
    my_katalk_data = list()
    
    # 정규 패턴 정의
    # --------------- 2023년 3월 21일 화요일 ---------------
    # date_info = r"[0-9]{4}년 [0-9]{1,2}월 [0-9]{1,2}일 \S요일"

    # [user1] [오전 11:22] 반갑습니다 :) 저는 제니퍼입니당!
    katalk_msg_pattern = "\[.*\] \[오\S [0-9]{1,2}:[0-9]{1,2}\].*"

    # 파일 데이터 읽기
    cnt = 0
    msg_date = ''
    f = open(file_path, encoding='utf-8')    
    for line in f :
        cnt += 1
        
        # 메시지 일시 패턴 : --------------- 2023년 3월 21일 화요일 ---------------
        if re.match("--------------- ", line):   
            line = line.split('--------------- ')
            line = line[1].split("일")              # 년월일
            date_time = line[0]+'일'
        
        # 카카오톡 메시지 패턴 : [user1] [오전 11:22] 반갑습니다 :) 저는 제니퍼입니당!
        elif re.match(katalk_msg_pattern, line):   
            p_pattern = r'\[([^\]]+)\]'             # 대괄호 패턴
            p = re.findall(p_pattern, line)
            user_name = p[0]                        # 메시지 보낸 사람
            msg_time = p[1]                         # 시간
            line = line.split(" ")
            text = ' '.join(line[3:])               # 메시지
            # DataFrame으로 나타내기 위해 my_katalk_data 리스트에 딕셔너리 타입으로 추가하기
            my_katalk_data.append({'date_time': date_time + ' ' + msg_time,
                                   'user_name': user_name,
                                   'text': text
                                   })
    print(f'파일 read count: {cnt}')
    
    # DataFrame 데이터로 만들기
    my_katalk_df = pd.DataFrame(my_katalk_data)
    print(f'DataFrame count: {len(my_katalk_df)}')
    return my_katalk_df


df = katalk_msg_parse_iphone(file)

df.head()

-----------------------

### 3. 데이터 전처리(컬럼 분할)

#### 3.1 컬럼 추가하기

In [ ]:
df['datetime'] = df['date_time']

#### 3.2 컬럼 순서 변경하기

In [ ]:
df.columns
df = df[ ['date_time', 'datetime', 'user_name', 'text'] ]

#### 3.3  날짜, 시간 세분화하기

In [ ]:
# 오전/오후 --> AM/PM 으로 변경
df['datetime'] = df['datetime'].str.replace('오전', 'AM')
df['datetime'] = df['datetime'].str.replace('오후', 'PM')
df.head(3)

In [ ]:
# pandas 표준 date_time 포맷으로 변경
# df['datetime'] = pd.to_datetime(df['datetime'], format='%Y. %m. %d. %p %I:%M')
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y년 %m월 %d일 %p %H:%M')
df.head(3)

In [ ]:
# text	date	year	month	day	weekday	hour 컬럼 추가
df['date'] = df['datetime'].dt.date
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['weekday'] = df['datetime'].dt.day_name()
df['hour'] = df['datetime'].dt.hour
df.head(3)

#### 3.4 카카오톡 메시지 길이

In [ ]:
# 메시지 길이, 메시지 단어 개수  컬럼 추가
df['msg_len'] = df['text'].str.len()  # 메시지 길이
df['msg_word_count'] = df['text'].str.split().str.len() #메시지 단어 개수
df.head()

#### 3.5  사진, 동영상 정보 추출

In [ ]:
audio_visual_text = '^동영상$|^사진$|^사진 [0-9]{1,2}장$'
mask = df['text'].str.contains(audio_visual_text)
df.loc[mask, 'audio_visual'] = 1
df.loc[~mask, 'audio_visual'] = 0

In [ ]:
# 메시지 건수 조정 (동영상/사진 등을 메시지로 인식한 경우 메시지 건수 0)
df.loc[mask, 'msg_len'] = 0
df.loc[mask, 'msg_word_count'] = 0
df[df['audio_visual']==1]

#### 3.6  통화 정보 추출하기
- 페이스톡 해요 (Face Call) : 통화 걸기
- 페이스톡 취소 (Cancelled Face Call): 통화 취소
- 페이스톡 부재중 (Missed Video Call): 상대방이 걸어왔지만 내가 응답 없었을 때
- 페이스톡 응답없음 (No Answer on Video Call): 내가 걸었지만 상대방이 응답 없었을 때
- 페이스톡 8:02 (Video Call 8:02): 10분 이내 통화
- 페이스톡 26:25 (Video Call 26:25): 10분 이상 통화
- 페이스톡 1:30:19 (Video Call 1:30:19): 1시간 이상 통화

In [ ]:
df[df['text'].str.contains('보이스톡|페이스톡')]

In [ ]:
def get_call_length(call_df):

    call_mask = call_df.text.str.contains('해요|취소|부재중|응답없음')  # 0
    call_df = call_df.text.str.extract('([0-9]{1,2}:)*([0-9]{1,2}):([0-9]{2})')  # 1
    call_df[0] = call_df[0].str.replace(':', '')  # 2
    call_df[0] = call_df[0].fillna(0)  # 3
    call_df = call_df.astype(float)  # 4
    call_df['call_len'] = call_df[0] * 3660 + call_df[1] * 60 + call_df[2]  # 5
    call_df.loc[call_mask, 'call_len'] = 0  # 6

    return call_df[['call_len']]  # 7

call_col_dict = {
    '페이스톡|Video Call': 'facetalk',
    '보이스톡|Voice Call': 'voicetalk',
}


for call in call_col_dict:
    mask = df['text'].str.contains(call)
    call_df = get_call_length(df[mask])
    call_df = call_df.rename(columns={'call_len': call_col_dict[call]})
    call_df.head()

    df = pd.concat([df, call_df], axis=1)

In [ ]:
mask = (df['voicetalk'] >= 0) | (df['facetalk'] >= 0)
df.loc[mask, 'msg_len'] = 0
df.loc[mask, 'msg_word_count'] = 0

In [ ]:
df[df['text'].str.contains('보이스톡|페이스톡')]

#### 3.7  비언어 표현 추출하기(이모티콘 등)

| 비언어 표현 | 텍스트 데이터 예시 | 정규표현식 |
|--|--|--|
|웃음, 울음 (ㅋㅋ, ㅎㅎ)  |ㅋㅋ, ㅋㅋㅋㅋ, ㅎㅎㅎㅎㅎ  |[ㅋㅎㅠㅜ]+  |
|느낌표 (!) , 물음표 (?)  |!, !!!!!!!, !?!?, ?????  |[!?~]+  |
| 쉼표(,), 점(.) | ……. ,,,,,, | [,.]{2,} |
| 이모티콘 타입 1 (입 중심)  | :) :D | 	[;:]{1}-?[)(DPpboOX] |
 |이모티콘 타입 1 (눈 중심)  | ^^, ㅜㅜ | ([>ㅜㅠㅡ@\^-+][ㅁㅇ0oO._-] *[\^ㅜㅠㅡ@<-+<];) |
| 이모티콘 타입 2 | 😃😆😍❤ | 정규표현식 대신 emoji 라이브러리 사용 |
| 이모티콘 타입 3 | (반함) (굿) (찡긋) (이모티콘 |  ((.*?)+) |

In [ ]:
df

In [ ]:
# 모두 포함하는 정규 표현식
# (\(.+?\)) | ([ㅋㅎㅠㅜ!?~]+) | [,.]{2,} | ([;:]{1}[\^\'-]?[)(DPpboOX]) | ([>ㅜㅠㅡ@+\^][ㅁㅇ0oO\._\-]*[\^ㅜㅠㅡ@+<];*)

In [ ]:
# 이모지 라이브러리 설치
!pip install emoji

In [ ]:
import emoji 

def extract_emojis(text):
    emoji_list = list()
    for c in text:
#         if c in emoji.UNICODE_EMOJI['en']:
        if c in emoji.EMOJI_DATA:
            emoji_list.append(c)

    return emoji_list

mimetic= "[ㅋㅎㅠㅜ!?~]+"
punctuations = "[,.]{2,}"
emo_type1_facial1 = "[;:]{1}[\^\'-]?[)(DPpboOX]"
emo_type1_facial2 = "[>ㅜㅠㅡ@\^][ㅁㅇ0oO\._\-]*[\^ㅜㅠㅡ@<];*"
emo_type3 = "\(.+?\)"

# 1개ㅔ 이상의 비언어 표현을 가지고 있는 데이터 출력하기
nonverbal_list = [mimetic, punctuations, emo_type1_facial1, emo_type1_facial2, emo_type3]

df['nonverbal'] = df['text'].str.findall('|'.join(nonverbal_list)) + df['text'].map(extract_emojis)
df['nonverbal_count'] = df['nonverbal'].apply(len)
df.loc[df['nonverbal_count'] > 0]


#### 이모티콘 사용

In [ ]:
# 이모티콘 사용한 레코드 변경
df.loc[df['text'] == '이모티콘', 'nonverbal'] = '[emoticon]'
df.loc[df['text'] == '이모티콘', 'nonverbal_count'] = 1
df.loc[df['text'] == '이모티콘', 'msg_len'] = 0
df.loc[df['text'] == '이모티콘', 'msg_word_count'] = 0
df.loc[df['text'] == '이모티콘']

In [ ]:
df.loc[df['text'] == '이모티콘'].count()

#### 3.8 URL 추출하기

In [ ]:
# URL 추출 라이브러리 설치하기
!pip install urlextract

In [ ]:
# URL 추출 예제
from urlextract import URLExtract

text = """Google Colab: https://colab.research.google.com. 
       Introduction for google colab: https://colab.research.google.com/notebooks/intro.ipynb"""

extractor = URLExtract()
urls = extractor.find_urls(text)

for url in urls:
    print(url)

In [ ]:
from urlextract import URLExtract


extractor = URLExtract()

df['url'] = df['text'].apply(extractor.find_urls)


df['url_count'] = df['url'].apply(len)
url_df = df.loc[df['url_count'] > 0]
url_df[['date_time','user_name','url','url_count']]


#### 3.9  특정 조건의 컬럼 값 변경

In [ ]:
df.loc[df['user_name'] == 'TMD-최수경-샘', 'user_name'] = '최수경'

#### 3.10  데이터 저장하기

In [ ]:
fname = './data/KakaoTalkChats_데이터분석_결과.csv'
df.to_csv(fname)

In [ ]:
df = pd.read_csv(fname, encoding='utf-8')
df

----------------------------------------

### 4. (기술통계)데이터 시각화
- https://wikidocs.net/162798

#### 4.1 일자별 선/막대 그래프 (plotly)

In [ ]:
import plotly.express as px

# 선그래프
fig = px.line(df, x='date', title='카카오톡 일자별 메시지')
fig

In [ ]:
# 막대 그래프
fig = px.bar(df, x='date', title='카카오톡 일자별 메시지')
fig

In [ ]:
# 점그래프
fig = px.scatter(df, x='date', title='카카오톡 일자별 메시지')
fig

#### 4.2 사용자별 메시지 건수

In [ ]:
# 유니크한 사용자
df['user_name'].unique()

In [ ]:
# 사용자별 메시지 건수(default  파레토(Pareto) 분포: 내림차순 정렬)
df['user_name'].value_counts()

In [ ]:
# 사용자별 메시지 건수
fig = px.bar(df['user_name'].value_counts())
fig.show()

#### 4.3 요일별 / 시간별 메시지 건수

In [ ]:
# 요일별 메시지 건수
df['weekday'].value_counts()

In [ ]:
# 시간별 메시지 건수
df['hour'].value_counts()

In [ ]:
# 시간별 메시지 건수(인덱스 순서대로 정렬)
df['hour'].value_counts().sort_index()

In [ ]:
# 시간별 메시지 건수
fig = px.bar(df['hour'].value_counts().sort_index())
fig.show()

#### 4.4 일자별 히스토그램 : 슬라이더 그래프

In [ ]:
# 일자별 히스토그램
fig = px.histogram(df, x='date')
fig.show()

In [ ]:
# 슬라이더 그래프 생성: 확대해서 보여준다.
fig = px.histogram(df, x='date')

fig.update_xaxes(   
    rangeslider_visible=True,
)
fig.show()

In [ ]:
# 사용자별 히스토그램
fig = px.histogram(df, x='date', color='user_name')
fig.show()

------------

THE END